In [12]:
import numpy as np
import sympy as sp

In [13]:
# 测试数据点
x = np.array([[3, 4, 1], [3, 3, 1]])
y = np.array([1, 1, -1])

In [19]:
N, M = x.shape
w = np.ones(M)
b = 0.0
# 定义变量
w1, w2, B, lam1, lam2, lam3 = sp.symbols('w_1 w_2 B lam1 lam2 lam3')
# 最小函数
f = (w1**2 + w2**2) / 2
# 约束条件
st1 = 1 - y[0] * (w1*3 + w2*3 + B)
st2 = 1 - y[1] * (w1*4 + w2*3 + B)
st3 = 1 - y[2] * (w1*1 + w2*1 + B)
# 定义拉格朗日乘子式
lagrange = f + lam1*st1 + lam2*st2 + lam3*st3
lagrange

lam1*(-B - 3*w_1 - 3*w_2 + 1) + lam2*(-B - 4*w_1 - 3*w_2 + 1) + lam3*(B + w_1 + w_2 + 1) + w_1**2/2 + w_2**2/2

In [20]:
# 定义kkt条件

# lam*st == 0
lam_st_kkts = [     
    lam1 * st1,
    lam2 * st2,
    lam3 * st3
]

# lam >= 0
lam_kkts = [
    sp.Ge(lam1, 0),
    sp.Ge(lam2, 0),
    sp.Ge(lam3, 0)
]

# st <= 0
st_kkts = [
    sp.Le(st1, 0),
    sp.Le(st2, 0),
    sp.Le(st3, 0)
]

lam_st_kkts

[lam1*(-B - 3*w_1 - 3*w_2 + 1),
 lam2*(-B - 4*w_1 - 3*w_2 + 1),
 lam3*(B + w_1 + w_2 + 1)]

In [6]:
# 求导
d_w1 = sp.diff(lagrange, w1)
d_w2 = sp.diff(lagrange, w2)
d_b = sp.diff(lagrange, B)

# 解方程
solve = sp.solve([d_w1, d_w2, d_b, *lam_st_kkts], [w1, w2, B, lam1, lam2, lam3])
[d_w1, d_w2, d_b, *lam_st_kkts], solve


([-3*lam1 - 4*lam2 + lam3 + w1,
  -3*lam1 - 3*lam2 + lam3 + w2,
  -lam1 - lam2 + lam3,
  lam1*(-B - 3*w1 - 3*w2 + 1),
  lam2*(-B - 4*w1 - 3*w2 + 1),
  lam3*(B + w1 + w2 + 1)],
 [(0, 0, -1, 0, 0, 0),
  (0, 0, B, 0, 0, 0),
  (0, 1, -2, 3/2, -1, 1/2),
  (6/13, 4/13, -23/13, 0, 2/13, 2/13),
  (1/2, 1/2, -2, 1/4, 0, 1/4)])

In [7]:
# 遍历结果，找到使f最小的解

min_solve = None
min_subs = 1e20

for i in solve:
    # 将单独的一个解封装成字典
    i = {
        'w1' : i[0],
        'w2' : i[1],
        'B' : i[2],
        'lam1' : i[3],
        'lam2' : i[4],
        'lam3' : i[5]
    }
    # 检查是否符合kkt条件，如果不符合去掉
    if not lam_kkts[0].subs(i):
        continue
    if not lam_kkts[1].subs(i):
        continue
    if not lam_kkts[2].subs(i):
        continue

    if st_kkts[0].subs(i) != True:
        continue
    if st_kkts[1].subs(i) != True:
        continue
    if st_kkts[2].subs(i) != True:
        continue

    # 找到使f得到最小值的解
    subs = f.subs(i)
    if subs < min_subs:
        min_subs = subs
        min_solve = i
min_solve

# 上述就是一个完整的硬间隔支持向量机的简单实现

{'w1': 1/2, 'w2': 1/2, 'B': -2, 'lam1': 1/4, 'lam2': 0, 'lam3': 1/4}

In [11]:
# 使用对偶问题解决支持向量机

# 定义符号
w1, w2, B, lam1, lam2, lam3 = sp.symbols('w1 w2 B lam1 lam2 lam3')

# 组装对偶式中的双层累加部分
sigma = 0
lams = [lam1, lam2, lam3]
for i in range(N):
    for j in range(N):
        sigma += lams[i] * lams[j] * y[i] * y[j] * x[i].dot(x[j])
sigma

26*lam1**2 + 44*lam1*lam2 + 19*lam2**2

In [16]:
# 对偶式
f = sigma / 2 - sum(lams)
f 

13*lam1**2 + 22*lam1*lam2 - lam1 + 19*lam2**2/2 - lam2 - lam3

In [15]:
# 约束：和等于0
st = sum(lams*y)
st

lam1 + lam2 - lam3

In [17]:
# 根据对b的导函数等于0可以直接替换lam3
f = f.subs(lam3, lam1 + lam2)
f

13*lam1**2 + 22*lam1*lam2 - 2*lam1 + 19*lam2**2/2 - 2*lam2

In [23]:
# 求导
d_lam1 = sp.diff(f, lam1)
d_lam2 = sp.diff(f, lam2)

# 解方程, 所有的lambda都应该大于0
solve = sp.solve([d_lam1, d_lam2], lam1, lam2)
solve

{lam1: -3/5, lam2: 4/5}

In [30]:
# 把lambda2 == 0带入方程
f_lam2_is_0 = f.subs(lam2, 0)
# 计算lam1
solve = sp.solve([sp.diff(f_lam2_is_0, lam1)], lam1)
solve[lam2] = 0

# 带入式子计算
subs = f.subs(solve)
subs

-1/13

In [32]:
# 把lambda1 == 0带入方程
f_lam1_is_0 = f.subs(lam1, 0)
# 计算lam2
solve = sp.solve([sp.diff(f_lam1_is_0, lam2)], lam2)
solve[lam1] = 0

# 带入式子计算
subs = f.subs(solve)
subs

-2/19

In [34]:
# 组装成numpy
solves = np.array([
    float(solve[lam1]),
    float(solve[lam2]),
    float(solve[lam1] + solve[lam2])
])
solves

array([0.        , 0.10526316, 0.10526316])

In [47]:
# 求w
w = np.dot(solves * y, x.T)
w

# 求b
b = y[0] - np.dot(solves * y, np.dot(x[0], x))

ValueError: shapes (3,) and (2,3) not aligned: 3 (dim 0) != 2 (dim 0)